# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with: mongoimport --db uk_food --collection establishments --file establishments.json --jsonArray


In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
# Connect to the default host and port
client = MongoClient('mongodb://localhost:27017/')

# Database name
db_name = 'uk_food'

# Collection name
collection_name = 'establishments'

# Check if the database exists
db_list = client.list_database_names()
if db_name in db_list:
    print(f"Database '{db_name}' found.")

Database 'uk_food' found.


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [7]:
# review the collections in our new database
if collection_name in db.list_collection_names():
    print(f"Collection '{collection_name}' found.")
else:
    print(f"Collection '{collection_name}' not found.")

Collection 'establishments' found.


In [18]:
# review a document in the establishments collection
one_document = db[collection_name].find_one()
pprint(one_document)


{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65fcb78ae582df888fc4007f'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [19]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [20]:
# Create a dictionary for the new restaurant data

new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [22]:
# Select the database
db = client['uk_food']

# Select the collection
collection = db['establishments']

# Insert the new restaurant into the collection
insert_Penang = collection.insert_one(new_restaurant_data)

In [23]:
# Check that the new restaurant was inserted
print(insert_Penang)

InsertOneResult(ObjectId('65fce1365d6f9c8651f1b58b'), acknowledged=True)


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [24]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = "Restaurant/Cafe/Canteen"
query = {"BusinessType": business_type}

# Project only the BusinessTypeID and BusinessType fields
projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

# Perform the query
x = db[collection_name].find_one(query, projection)

# Print the result
print(x)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [25]:
# Update the new restaurant with the correct BusinessTypeID
# Define the update data
update_data = {'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}

# Update the document named 'insert_Penang'
query = {'Name': 'insert_Penang'}  # Assuming 'Name' field uniquely identifies the document
result = db[collection_name].update_one(query, {'$set': update_data})

In [26]:
# Confirm that the new restaurant was updated
print(insert_Penang)

InsertOneResult(ObjectId('65fce1365d6f9c8651f1b58b'), acknowledged=True)


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [30]:
# Find how many documents have LocalAuthorityName as "Dover"
# Define the query to count documents with LocalAuthorityName as "Dover"
query = {"LocalAuthorityName": "Dover"}

# Count the documents
count = db[collection_name].count_documents(query)

# Print the count
print(f"Number of documents with LocalAuthorityName as 'Dover': {count}")

Number of documents with LocalAuthorityName as 'Dover': 994


In [31]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = db[collection_name].delete_many(query)

In [33]:
# Check if any remaining documents include Dover
query = {"LocalAuthorityName": "Dover"}

# Count the documents
count = db[collection_name].count_documents(query)

# Print the count
print(f"Number of documents with LocalAuthorityName as 'Dover': {count}")

Number of documents with LocalAuthorityName as 'Dover': 0


In [34]:
# Check that other documents remain with 'find_one'
document = db[collection_name].find_one({})

# Print the retrieved document
print("Retrieved document:")
print(document)

Retrieved document:
{'_id': ObjectId('65fcb78ae582df888fc40366'), 'FHRSID': 1043695, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000073616', 'BusinessName': 'The Pavilion', 'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1, 'AddressLine1': 'East Cliff Pavilion', 'AddressLine2': 'Wear Bay Road', 'AddressLine3': 'Folkestone', 'AddressLine4': 'Kent', 'PostCode': 'CT19 6BL', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2018-04-04T00:00:00', 'LocalAuthorityCode': '188', 'LocalAuthorityName': 'Folkestone and Hythe', 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk', 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.195625', 'latitude': '51.083812'}, 'RightToReply': '', 'Distance': 4591.765489457773, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:0

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [44]:
db = client['uk_food']
collection_name = 'establishments'

# Change the data type from String to Decimal for longitude and latitude
update_operation = [
    {
        '$set': {
            'geocode.latitude': {'$toDouble': '$geocode.latitude'},
            'geocode.longitude': {'$toDouble': '$geocode.longitude'}
        }
    }
]

# Execute the update operation with update_many()
update_result = db[collection_name].update_many({}, update_operation)

Use `update_many` to convert `RatingValue` to integer numbers.

In [45]:
db = client['uk_food']
collection_name = 'establishments'

# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])
update_operation = [
    {
        '$set': {
            'RatingValue': {'$toInt': '$RatingValue'}
        }
    }
]

# Execute the update operation with update_many()
update_result = db[collection_name].update_many({}, update_operation)

In [46]:
db = client['uk_food']
collection_name = 'establishments'

# Change the data type from String to Integer for RatingValue
update_operation = [
    {
        '$set': {
            'RatingValue': {'$toInt': '$RatingValue'}
        }
    }
]

# Execute the update operation with update_many()
update_result = db[collection_name].update_many({}, update_operation)

In [43]:
import pandas as pd

db = client['uk_food']
collection_name = 'establishments'

# Check that the coordinates and rating value are now numbers
df = pd.DataFrame(list(db[collection_name].find()))

# Check the data types of the columns
print("Data types of the columns:")
print(df.dtypes)

Data types of the columns:
_id                            object
FHRSID                        float64
ChangesByServerID             float64
LocalAuthorityBusinessID       object
BusinessName                   object
BusinessType                   object
BusinessTypeID                 object
AddressLine1                   object
AddressLine2                   object
AddressLine3                   object
AddressLine4                   object
PostCode                       object
Phone                          object
RatingValue                   float64
RatingKey                      object
RatingDate                     object
LocalAuthorityCode             object
LocalAuthorityName             object
LocalAuthorityWebSite          object
LocalAuthorityEmailAddress     object
scores                         object
SchemeType                     object
geocode                        object
RightToReply                   object
Distance                      float64
NewRatingPending       